In [2]:
import pandas as pd
import os

import explore

In [3]:
nbastats, nbapo, nbadata, nbastats_po, pbpstats, shotdetail, shotdetail_po = explore.import_nba()

In [4]:
df = [nbastats, nbapo, nbadata, nbastats_po, 
      pbpstats, shotdetail, shotdetail_po]

# Which features to keep?

To begin I will just be keeping score by first quarter, half, 3rd quarter and final.

Each test will be in the next sequence. The model will try all teh games in the past, and the last 5 to ten games. As well as their last games. 

### Features

team_id
home or away
opponent_id
record total
record 
total points scored in the event
wins over last 5
wins over last game
wins over last 10


##### offensive stats
points
points first quarter
points second quarter
points third quarter
field goal percentage
field goal pts
three point pts
three point percentage
shooting fouls drawn
free throws
free throw percentage
assists


##### defensive stats
blocks 
steals 
turnovers 
rebounds
personal fouls


#### like to have
starting 5
- number of points
- complete prodution broken down by player
bench player
height of starting 5

In [12]:
shotdetail.shape

(217218, 24)

In [6]:
nbastats.drop(columns=[''])

GAME_ID                           0
EVENTNUM                          0
EVENTMSGTYPE                      0
EVENTMSGACTIONTYPE                0
PERIOD                            0
WCTIMESTRING                      0
PCTIMESTRING                      0
HOMEDESCRIPTION              276929
NEUTRALDESCRIPTION           562111
VISITORDESCRIPTION           280394
SCORE                        415034
SCOREMARGIN                  415034
PERSON1TYPE                       2
PLAYER1_ID                        0
PLAYER1_NAME                  47066
PLAYER1_TEAM_ID               47759
PLAYER1_TEAM_CITY             47759
PLAYER1_TEAM_NICKNAME         47759
PLAYER1_TEAM_ABBREVIATION     47759
PERSON2TYPE                       0
PLAYER2_ID                        0
PLAYER2_NAME                 385582
PLAYER2_TEAM_ID              385582
PLAYER2_TEAM_CITY            385582
PLAYER2_TEAM_NICKNAME        385582
PLAYER2_TEAM_ABBREVIATION    385582
PERSON3TYPE                       0
PLAYER3_ID                  

### fields that contain data i need

#### nbastats

#### nbadata
evt
de?
opt1 -?
opt2 -?
opt3 -?
opt4 -?
tid
hs
vs
epid - ?
oftid
ord -?
period
game - id

###### drop
wallclk
cl
locx
locy


#### pbpstats
events
fg2a
fg2m
fg3a
fg3m
gamedate -?
gameid
OFFENSIVEREBOUNDS
opponent
period
SHOOTINGFOULSDRAWN
TURNOVERS
DESCRIPTION -?


##### drop
endtime
NONSHOOTINGFOULSTHATRESULTEDINFTS
OFFENSIVEREBOUNDS
STARTSCOREDIFFERENTIAL
STARTTIME
STARTTYPE
URL


#### datanba
de -?
opt1
opt2 -?
mtype -?
etype -?
opid -?
tid
pid
hs
vs
epid -?
oftid -?
period
game_id

##### drop
evt
wallclk
cl
locX
locY


#### shotdetail
game_id
TEAM_ID
TEAM_NAME
PERIOD
EVENT_TYPE
ACTION_TYPE
SHOT_TYPE
HTM	

##### drop
grid_type
game_event_id
player_id
PLAYER_NAME
MINUTES_REMAINING
SECONDS_REMAINING
SHOT_ZONE_BASIC
SHOT_ZONE_AREA
SHOT_ZONE_RANGE
SHOT_DISTANCE
LOC_X
LOC_Y
SHOT_ATTEMPTED_FLAG
SHOT_MADE_FLAG
GAME_DATE
VTM